## Generative Adversarial Networks (GANs) using PyTorch

In the following demos we will explore, each technique (GANs, VAEs, and Diffusion Models) by its own code since each one of them involves distinct architectures and training procedures. 

Below is a basic outline and simplified implementation for each approach using popular libraries like TensorFlow/Keras and PyTorch. These examples are rudimentary and meant for educational purposes to provide a starting point for deeper exploration.

## Recap

GANs consist of two neural networks:

1. **Generator:** Generates fake data from random noise.
2. **Discriminator:** Tries to distinguish between real data and data generated by the Generator.

The Generator and Discriminator are trained together in a way that the Generator gets better at creating realistic data, while the Discriminator gets better at telling the real data apart from the fake dat

### Import Libraries

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.utils import save_image
from torchvision import datasets, transforms
from torch.utils.data import DataLoader


### Setup variables

In [ ]:
# Hyperparameters
latent_dim = 100
batch_size = 64
lr = 0.0002
epochs = 50

### Create the Generator and Discriminator functions

In [ ]:
# Generator Network
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(latent_dim, 128),
            nn.ReLU(True),
            nn.Linear(128, 256),
            nn.ReLU(True),
            nn.Linear(256, 512),
            nn.ReLU(True),
            nn.Linear(512, 1024),
            nn.ReLU(True),
            nn.Linear(1024, 28*28),
            nn.Tanh()
        )

    def forward(self, x):
        return self.model(x).view(x.size(0), 1, 28, 28)
    

# Discriminator Network
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = x.view(x.size(0), -1)
        return self.model(x)

### Initialize the networks

In [ ]:
# Initialize networks
generator = Generator()
discriminator = Discriminator()

### Setup the loss and optimizers

In [ ]:
# Loss and optimizers
criterion = nn.BCELoss()
optimizer_g = optim.Adam(generator.parameters(), lr=lr)
optimizer_d = optim.Adam(discriminator.parameters(), lr=lr)

### Setup the data

In [ ]:
# Data loader
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize([0.5], [0.5])])
mnist = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
dataloader = DataLoader(mnist, batch_size=batch_size, shuffle=True)

### Implement the training loop

In [ ]:
# Training loop
for epoch in range(epochs):
    for i, (imgs, _) in enumerate(dataloader):
        real = torch.ones(imgs.size(0), 1)
        fake = torch.zeros(imgs.size(0), 1)

        # Train Discriminator
        optimizer_d.zero_grad()
        outputs = discriminator(imgs)
        loss_real = criterion(outputs, real)
        z = torch.randn(imgs.size(0), latent_dim)
        fake_imgs = generator(z)
        outputs = discriminator(fake_imgs.detach())
        loss_fake = criterion(outputs, fake)
        loss_d = loss_real + loss_fake
        loss_d.backward()
        optimizer_d.step()

        # Train Generator
        optimizer_g.zero_grad()
        outputs = discriminator(fake_imgs)
        loss_g = criterion(outputs, real)
        loss_g.backward()
        optimizer_g.step()

    print(f'Epoch [{epoch+1}/{epochs}], Loss D: {loss_d.item():.4f}, Loss G: {loss_g.item():.4f}')

    if epoch % 10 == 0:
        save_image(fake_imgs.data[:25], f'images_{epoch}.png', nrow=5, normalize=True)